In [ ]:
# 7.1 - Implement a Recurrent Neural Network (RNN) using Pytorch

# Define RNN model
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, rnn_type='RNN'):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if rnn_type == 'RNN':
          self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True) # (a) Build a one-layer RNN...
        elif rnn_type == 'LSTM':
          self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)  # (b) Build a linear classifier...
        self.rnn_type = rnn_type  # Store the RNN type

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        if self.rnn_type == 'LSTM':
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # Initialize cell state for LSTM
            out, _ = self.rnn(x, (h0, c0)) # LSTM returns (out, (hn, cn))
        else:
            out, _ = self.rnn(x, h0) # RNN returns (out, hn)
        logit = self.fc(out[:, -1, :])
        prob = nn.functional.softmax(logit, dim=1)
        return prob, logit

# fix seed
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# load data

data_dir = 'human+activity+recognition+using+smartphones/UCI HAR Dataset'
train_dataset = UCIHARDataset(subset='train', data_dir=data_dir)
test_dataset = UCIHARDataset(subset='test', data_dir=data_dir)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# hyperparams
input_size = 9
hidden_size = 16
num_layers = 1
num_classes = 6
num_epochs = 10
learning_rate = 0.001
batch_size = 16

criterion = nn.CrossEntropyLoss()

# helper func
def train_and_evaluate_rnn(model, train_loader, test_loader, num_epochs, optimizer, criterion):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            _, activity_logit = model(inputs)
            loss = criterion(activity_logit, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, _ = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = correct / total
    return test_acc

# 7.1.a - Build a one-layer RNN that read the inputs (input dimension=9) and produce hidden  features (use feature dimension=16).
# 7.1.b - Build a linear classifier that classifies the activity labels (number of activity=6) from hidden features.
# 7.1.c Train the model using Cross Entropy Loss and Adam optimizer.
# 7.1.d Train for 10 epochs with batch size=16, learning rate=0.001. Report test accuracy.
model_1_layer = RNNClassifier(input_size, hidden_size, num_layers, num_classes, rnn_type='RNN')
optimizer_1_layer = optim.Adam(model_1_layer.parameters(), lr=learning_rate)
test_acc_1_layer = train_and_evaluate_rnn(model_1_layer, train_loader, test_loader, num_epochs, optimizer_1_layer, criterion)
print("7.1.d Test Accuracy (10 epochs, 16 batch size, lr 0.0011 layer RNN, hidden_size=16):", test_acc_1_layer)

# 7.2 - Train and evaluate the model with different architectures and report test accuracy for each model:

# 7.2.a - Change feature dimension to 64
hidden_size_a = 64
model_a = RNNClassifier(input_size, hidden_size_a, num_layers, num_classes, rnn_type='RNN')
optimizer_a = optim.Adam(model_a.parameters(), lr=learning_rate)
test_acc_a = train_and_evaluate_rnn(model_a, train_loader, test_loader, num_epochs, optimizer_a, criterion)
print("7.2.a Test Accuracy (feature dimension=64:", test_acc_a)

# 7.2.b - Change the number of RNN layers to 2,3,4 (keep feature dimension=16)
for num_layers_b in [2, 3, 4]:
    model_b = RNNClassifier(input_size, hidden_size, num_layers_b, num_classes, rnn_type='RNN')
    optimizer_b = optim.Adam(model_b.parameters(), lr=learning_rate)
    test_acc_b = train_and_evaluate_rnn(model_b, train_loader, test_loader, num_epochs, optimizer_b, criterion)
    print("7.2.b Test Accuracy (", num_layers_b, "layer RNN, hidden_size=16):", test_acc_b)

# 7.2.c - Change RNN to LSTM (keep feature dimension=16 and number of layers=1)
model_c = RNNClassifier(input_size, hidden_size, 1, num_classes, rnn_type='LSTM')
optimizer_c = optim.Adam(model_c.parameters(), lr=learning_rate)
test_acc_c = train_and_evaluate_rnn(model_c, train_loader, test_loader, num_epochs, optimizer_c, criterion)
print("7.2.c Test Accuracy (1 layer LSTM, hidden_size=16):", test_acc_c)